# 🚀 Eventformer: Frame-Free Vision Transformer for Event Cameras

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/Eventformer/blob/main/Eventformer_Colab.ipynb)

This notebook allows you to train and evaluate Eventformer on Google Colab with free GPU.

**Key Components:**
- CTPE: Continuous-Time Positional Encoding
- PAAA: Polarity-Aware Asymmetric Attention
- ASNA: Adaptive Spatiotemporal Neighborhood Attention

## 1. Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Clone repository (replace YOUR_USERNAME with your GitHub username)
!git clone https://github.com/YOUR_USERNAME/Eventformer.git
%cd Eventformer/code

In [ ]:
# Install dependencies
!pip install -q einops timm h5py tensorboard seaborn

## 2. Test Installation

In [ ]:
# Run all component tests
!python main.py --mode test

## 3. Download Datasets

Choose the dataset you want to use:

In [ ]:
# Create data directories
!mkdir -p ../data/ncaltech101
!mkdir -p ../data/dvs128_gesture
!mkdir -p ../data/gen1

In [ ]:
# Option 1: Download N-Caltech101 (recommended for quick testing)
# Note: You may need to manually download from:
# https://www.garrickorchard.com/datasets/n-caltech101

# If you have the dataset on Google Drive:
# from google.colab import drive
# drive.mount('/content/drive')
# !cp -r /content/drive/MyDrive/datasets/ncaltech101/* ../data/ncaltech101/

In [ ]:
# Mount Google Drive (optional - for saving checkpoints and loading data)
from google.colab import drive
drive.mount('/content/drive')

# Create output directory in Drive
!mkdir -p /content/drive/MyDrive/Eventformer_outputs

## 4. Training

Train Eventformer on your chosen dataset.

In [ ]:
# Quick test with synthetic data (no download needed)
# The code generates synthetic events automatically when real data is not found

!python train.py \
    --dataset ncaltech101 \
    --data_root ../data \
    --model tiny \
    --epochs 10 \
    --batch_size 16 \
    --num_events 2048 \
    --output_dir ../outputs

In [ ]:
# Full training (when you have real data)
!python train.py \
    --dataset ncaltech101 \
    --data_root ../data \
    --model small \
    --epochs 100 \
    --batch_size 32 \
    --num_events 4096 \
    --lr 1e-4 \
    --output_dir /content/drive/MyDrive/Eventformer_outputs

## 5. Ablation Study

In [ ]:
# Run ablation study to evaluate component contributions
!python ablation.py \
    --dataset ncaltech101 \
    --data_root ../data \
    --model_size tiny \
    --epochs 20 \
    --num_runs 3 \
    --configs full no_ctpe no_paaa no_asna \
    --output_dir ../ablation_results

## 6. Evaluation

In [ ]:
# Evaluate trained model
# Replace with your actual checkpoint path
!python evaluate.py \
    --checkpoint ../outputs/best_model.pth \
    --dataset ncaltech101 \
    --data_root ../data \
    --benchmark

## 7. Generate Figures

In [ ]:
# Generate publication-quality figures
!python visualize.py --output_dir ../figures

# Display generated figures
from IPython.display import Image, display
import os

figures_dir = '../figures'
if os.path.exists(figures_dir):
    for fig in os.listdir(figures_dir):
        if fig.endswith('.png'):
            print(f"\n{fig}:")
            display(Image(os.path.join(figures_dir, fig)))

## 8. Interactive Model Testing

In [ ]:
import sys
sys.path.insert(0, '.')

import torch
from models import eventformer_tiny, eventformer_small, EventformerForClassification

# Create model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = eventformer_tiny().to(device)

# Print model info
num_params = sum(p.numel() for p in model.parameters())
print(f"Model: Eventformer-Tiny")
print(f"Parameters: {num_params:,} ({num_params/1e6:.2f}M)")
print(f"Device: {device}")

# Test forward pass
B, N = 4, 4096
coords = torch.rand(B, N, 2).to(device) * torch.tensor([346, 260]).to(device)
times = torch.rand(B, N).to(device).sort(dim=1)[0]
polarities = (torch.randint(0, 2, (B, N)).float() * 2 - 1).to(device)

with torch.no_grad():
    features, out_coords, out_times, out_pols = model(coords, times, polarities)

print(f"\nInput: {N} events per sample")
print(f"Output: {features.shape[1]} events, {features.shape[2]} features")

In [ ]:
# Speed benchmark
import time

model.eval()
num_iters = 100

# Warmup
for _ in range(10):
    with torch.no_grad():
        _ = model(coords, times, polarities)
torch.cuda.synchronize()

# Benchmark
start = time.perf_counter()
for _ in range(num_iters):
    with torch.no_grad():
        _ = model(coords, times, polarities)
torch.cuda.synchronize()
elapsed = time.perf_counter() - start

print(f"\nSpeed Benchmark:")
print(f"  Batch size: {B}")
print(f"  Events: {N}")
print(f"  Time per batch: {elapsed/num_iters*1000:.2f} ms")
print(f"  Throughput: {B*num_iters/elapsed:.1f} samples/sec")

## 9. Save Results to Google Drive

In [ ]:
# Copy results to Google Drive
!cp -r ../outputs/* /content/drive/MyDrive/Eventformer_outputs/ 2>/dev/null || echo "No outputs to copy"
!cp -r ../ablation_results/* /content/drive/MyDrive/Eventformer_outputs/ 2>/dev/null || echo "No ablation results to copy"
!cp -r ../figures/* /content/drive/MyDrive/Eventformer_outputs/ 2>/dev/null || echo "No figures to copy"

print("Results saved to Google Drive!")

---

## Notes

**GPU Runtime:**
- Go to Runtime → Change runtime type → Select GPU (T4 is free, A100 for Colab Pro)

**Dataset Download:**
- N-Caltech101: https://www.garrickorchard.com/datasets/n-caltech101
- DVS128 Gesture: https://research.ibm.com/interactive/dvsgesture/
- GEN1: https://www.prophesee.ai/2020/01/24/prophesee-gen1-automotive-detection-dataset/

**Memory Issues:**
- Reduce `--batch_size` or `--num_events` if you run out of GPU memory
- Use `--model tiny` instead of larger models

**Save Checkpoints:**
- Mount Google Drive to save checkpoints between sessions
- Use `--output_dir /content/drive/MyDrive/Eventformer_outputs`